In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import time
from ipywidgets import HTML
from ipyleaflet import Map, Marker, Popup, basemaps, AwesomeIcon
from IPython.display import display
import mysql.connector
from mysql.connector import errorcode
import configparser

config= configparser.ConfigParser()
config.read('config.ini')
# Load the necessary GTFS data files
data_path = config.get("app","data_path")
stops = pd.read_csv(f"{data_path}/stops.txt")
routes = pd.read_csv(f"{data_path}/routes.txt")
trips = pd.read_csv(f"{data_path}/trips.txt")
stop_times = pd.read_csv(f"{data_path}/stop_times.txt")
calendar = pd.read_csv(f"{data_path}/calendar.txt")

#Connect to the database
db = mysql.connector.connect(
    host=config.get("app","host"),
    user=config.get("app","user"),
    password=config.get("app","password"),
    database=config.get("app","database")
)
cursor = db.cursor()
insert_buses = ("INSERT INTO buses "
               "(route_short_name, trip_id, service_id, route_id, direction_id,arrival_time,departure_time,stop_id,stop_name,stop_sequence,stop_lat,stop_lon,update_time) "
               "VALUES (%s, %s,%s, %s, %s,%s,%s,%s,%s,%s,%s,%s,%s)")
update_rows=("UPDATE buses SET route_short_name=%s, service_id=%s, route_id=%s, direction_id=%s, arrival_time=%s, departure_time=%s, stop_id=%s, stop_name=%s, stop_sequence=%s, stop_lat=%s, stop_lon=%s, update_time=%s WHERE trip_id=%s ")
delete_buses=("DELETE FROM buses")
# Helper function to convert time strings to HH:MM:SS format
def convert_time(time_str):
    try:
        parts = time_str.split(':')
        hours, minutes, seconds = int(parts[0]), int(parts[1]), int(parts[2])
        if hours >= 24:
            hours -= 24
        return f"{hours:02}:{minutes:02}:{seconds:02}"
    except ValueError as e:
        print(f"Error converting time: {time_str} - {e}")
        return None

# Apply the time conversion
stop_times['arrival_time'] = stop_times['arrival_time'].apply(convert_time)
stop_times['departure_time'] = stop_times['departure_time'].apply(convert_time)
stop_times = stop_times.dropna(subset=['arrival_time', 'departure_time'])

# Define the interval in seconds for updates
update_interval = 30  # 30 seconds

def get_time_input(prompt):
    while True:
        try:
            time_str = input(prompt)
            time_obj = datetime.strptime(time_str, '%H:%M:%S').time()
            return time_str
        except ValueError:
            print("Invalid time format. Please enter time in HH:MM:SS format.")

# Function to get the service IDs for a given date
def get_service_ids_for_date(date):
    day_index = date.weekday()
    days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    day_column = days[day_index]
    service_ids = calendar[calendar[day_column] == 1]['service_id'].tolist()
    return service_ids
# Function to get the buses at the current time
def get_buses_at_time(target_time_str, service_ids):
    # Filter the stop_times DataFrame for the target time
    filtered_stop_times = stop_times[
        (stop_times['arrival_time'] == target_time_str) | 
        (stop_times['departure_time'] == target_time_str)
    ]
    
    if filtered_stop_times.empty:
        print("No new bus data available, retaining previous state.")
        return []

    # Merge with trips to filter by the given service IDs
    filtered_stop_times = pd.merge(filtered_stop_times, trips, on='trip_id', how='left')
    filtered_stop_times = filtered_stop_times[filtered_stop_times['service_id'].isin(service_ids)]
    
    if filtered_stop_times.empty:
        return []

    # Merge with routes and stops to get detailed bus information
    merged_df = pd.merge(filtered_stop_times, routes, on='route_id', how='left')
    merged_df = pd.merge(merged_df, stops, on='stop_id', how='left')
    
    bus_data = []
    for _, row in merged_df.iterrows():
        routeNumber = row['route_short_name']
        tripID = row['trip_id']
        routeID = row['route_id']
        stopName = row['stop_name']
        arrivalTime = row['arrival_time']
        departureTime = row['departure_time']
        serviceID = row['service_id']
        stopSequence = row['stop_sequence']
        directionID = row['direction_id']
        stopID = row['stop_id']
        stopLat = row['stop_lat']
        stopLon = row['stop_lon']
        print(
        f"Route: {routeNumber}, Trip ID: {tripID}, Stop: {stopID}, Arrival Time: {arrivalTime}, Departure Time: {departureTime}, "
        f"Stop Name: {stopName}, Direction ID: {directionID}, Stop Lat: {stopLat}, Stop Lon: {stopLon}, Route ID: {routeID}, Service ID: {serviceID}, Stop Sequence: {stopSequence}")
        bus_data.append((routeNumber, tripID, serviceID, routeID, directionID, arrivalTime, departureTime,
                         stopID, stopName, stopSequence, stopLat, stopLon))   
    return bus_data

def add_stop_markers(stops_df, mymap, stop_markers):
    for stop in stops_df.itertuples():
        marker = Marker(location=[stop.stop_lat, stop.stop_lon], draggable=False)
        popup_content = HTML()
        popup_content.value = f"Stop: {stop.stop_name}, Stop Number: {stop.stop_id}"
        popup = Popup(
            location=[stop.stop_lat, stop.stop_lon],
            child=popup_content,
            close_button=False,
            auto_close=False,
            close_on_escape_key=False
        )
        marker.popup = popup
        mymap.add_layer(marker)
        stop_markers[stop.stop_id] = marker
    return stop_markers

def create_bus_marker(bus_data):
    icon = AwesomeIcon(name='bus', marker_color='red', icon_color='white')
    marker = Marker(icon=icon, location=[bus_data['stop_lat'], bus_data['stop_lon']], draggable=False)
    return marker

def update_popup(marker, bus_data):
    popup_content = HTML()
    popup_content.value = f"Route Name: {bus_data[0]}, Trip ID: {bus_data[1]}, Stop Name: {bus_data[8]}, Arrival Time: {bus_data[5]}"
    marker.popup = Popup(
        location=marker.location,
        child=popup_content,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )
# Start the dynamic updating service
def dynamic_update_service(start_time_str, end_time_str):
    # Initialize start and end datetime
    today_date = datetime.now().date()
    start_time = datetime.strptime(start_time_str, '%H:%M:%S').time()
    end_time = datetime.strptime(end_time_str, '%H:%M:%S').time()
    
    start_datetime = datetime.combine(today_date, start_time)
    end_datetime = datetime.combine(today_date, end_time)
    
    # If end time is before start time, assume it is on the next day
    if end_time < start_time:
        end_datetime += timedelta(days=1)

    current_datetime = start_datetime
    previous_markers = {}
    markers = {}
    stop_markers = {}

    # Initialize the map
    mapcenter = (stops['stop_lat'].mean(), stops['stop_lon'].mean())
    mymap = Map(center=mapcenter, zoom=12, basemap=basemaps.OpenStreetMap.Mapnik)
    
    # Adding the stop markers and displaying the map initially
    stop_markers = add_stop_markers(stops, mymap, stop_markers)
    display(mymap)

    while current_datetime <= end_datetime:
       
        current_time_str = current_datetime.strftime('%H:%M:%S')
        print(f"\nCurrent Time: {current_time_str}")

        # Get service IDs for the current date
        current_service_ids = get_service_ids_for_date(current_datetime.date())
        buses = get_buses_at_time(current_time_str, current_service_ids)
       
        # Check if the buses list is empty, if so, keep the previous state
        if not buses:
            print("No new bus data available, retaining previous state.")

        else:
           
            try:
            # Insert each bus data row into the database
                for data in buses:
                    cursor.execute(insert_buses, (*data, current_datetime))

            # Commit the transaction to save the data in the database
                    db.commit()

            except mysql.connector.Error as err:
                print(f"Error: {err}")
                db.rollback()  # Rollback the transaction on error

             # trip_id, stop_lat, stop_lon
            for bus_data in buses:
                trip_id = bus_data[1]
                stop_lat, stop_lon = bus_data[10], bus_data[11]
    
                if trip_id in markers:
                    # Update existing marker location and popup
                    marker = markers[trip_id]
                    if marker.location != [stop_lat, stop_lon]:  # Only update if the location has changed
                        marker.location = [stop_lat, stop_lon]
                        update_popup(marker, bus_data)

                        try:

                            for data in buses:
                                cursor.execute(update_rows, (*data, current_datetime))
                                db.commit()
                        except mysql.connector.Error as err:
                                print(f"Error: {err}")
                                db.rollback()  # Rollback the transaction on error   
                                     
                else:
                    # Create and add new marker for the bus
                    new_marker = create_bus_marker({
                        'stop_lat': stop_lat,
                        'stop_lon': stop_lon,
                        'route_short_name': bus_data[0],
                        'trip_id': bus_data[1],
                        'stop_name': bus_data[8],
                        'arrival_time': bus_data[5]
                    })
                    update_popup(new_marker, bus_data)
                    mymap.add_layer(new_marker)
                    markers[trip_id] = new_marker
                    previous_markers[trip_id] = (stop_lat, stop_lon)
            
        
        # Remove buses that have completed their stop sequence
        bus_trip_ids = [bus[1] for bus in buses]
        for trip_id in list(markers.keys()):
            if trip_id not in bus_trip_ids:
                mymap.remove_layer(markers[trip_id])
                del markers[trip_id]
                del previous_markers[trip_id]  # Also remove from previous_markers
                print(f"Trip ID {trip_id} has completed its trip and has been removed.")
        current_datetime += timedelta(seconds=update_interval)
        time.sleep(3)  # Reduce sleep time for faster testing

# Run the dynamic updating service with the specified start and end times
start_time_str = get_time_input("Enter start time (HH:MM:SS): ")
end_time_str = get_time_input("Enter end time (HH:MM:SS): ")
dynamic_update_service(start_time_str, end_time_str)
